##### Copyright 2025 Google LLC.

In [2]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 🚀 Memory Management - Part 1 - Sessions

**Welcome to Day 3 of the Kaggle 5-day Agents course!**

In this notebook, you'll learn:

- ✅ What sessions are and how to use them in your agent
- ✅ How to build *stateful* agents with sessions and events
- ✅ How to persist sessions in a database
- ✅ Context management practices such as context compaction
- ✅ Best practices for sharing session State

## ‼️ Please Read


> ❌ **ℹ️ Note: No submission required!**
> This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

> ⏸️ **Note:**  When you first start the notebook via running a cell you might see a banner in the notebook header that reads **"Waiting for the next available notebook"**. The queue should drop rapidly; however, during peak bursts you might have to wait a few minutes.

> ❌ **Note:** Avoid using the **Run all** cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order - one at a time. [See FAQ on 429 errors for more information.](https://www.kaggle.com/code/kaggle5daysofai/day-0-troubleshooting-and-faqs)

**For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.**

## 📖 Get started with Kaggle Notebooks

If this is your first time using Kaggle Notebooks, welcome! You can learn more about using Kaggle Notebooks [in the documentation](https://www.kaggle.com/docs/notebooks).

Here's how to get started:

**1. Verify Your Account (Required)**

To use the Kaggle Notebooks in this course, you'll need to verify your account with a phone number.

You can do this in your [Kaggle settings](https://www.kaggle.com/settings).

**2. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

This creates a private copy of the notebook just for you.

**3. Run Code Cells**

Once you have your copy, you can run code.

Click the ▶️ Run button next to any code cell to execute it.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

Run the cells in order from top to bottom.

**4. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.

---
## ⚙️ Section 1: Setup

### 1.1: Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

### 1.2: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [4]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Helper functions

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```


**Simple bhasha mein:** Ye function aapko AI se baat karne ka ek complete system deta hai - chahe ek question ho ya multiple questions ek saath.

In [5]:
# 'async' ka matlab hai "yeh function time lene wala kaam karega"
# Jaise: AI se baat karna, database se data lena - yeh sab time leta hai

async def run_session(  # 'async' = "Yeh function rukega (await) nahi, background mein kaam karega"
    runner_instance: Runner,
    user_queries: list[str] | str = None,  # User ke questions - ek ya multiple
    session_name: str = "default",  # Conversation session ka naam
):
    print(f"\n ### Session: {session_name}")

    # Runner se app ka naam le rahe hain
    app_name = runner_instance.app_name

    # Naya session banate hain ya purana wala retrieve karte hain
    try:
        # 'await' = "Ruko, session ban jaye tabhi aage badho"
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        # 'await' = "Ruko, existing session mil jaye tabhi aage badho"  
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Agar user ne koi questions diye hain toh process karte hain
    if user_queries:
        # Single question ko list mein convert karte hain
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Har question ko ek ke baad ek process karte hain
        for query in user_queries:
            print(f"\nUser > {query}")

            # Simple string ko ADK ke format mein convert karte hain
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # 'async for' = "AI ka response real-time mein aata rahega, har chunk ke saath kaam karo"
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check karte hain kya response mein kuch valid content hai
                if event.content and event.content.parts:
                    # Empty ya "None" responses ko skip karte hain
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("Koi questions nahi diye!")

print("✅ Helper functions define ho gayi hain.")

✅ Helper functions define ho gayi hain.


### 1.5: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

**🤹 Section 2: Session Management**

### 2.1 The Problem

Asal mein, LLMs (Large Language Models) **bilkul stateless hote hain**. Unhe sirf wohi cheez pata hoti hai jo aap current API call mein dete ho. Matlab, agar aap koi aisa agent banaye jisme context management na ho, toh woh har naye prompt ko bilkul naye sawal ki tarah treat karega - pichli baatcheet bhool jayega.

**❓ Problem samjhein?** Sochiye aap kisi aise insaan se baat kar rahe ho jo har sentence ke baad sab kuch bhool jata hai. Bilkul waisi hi situation hai raw LLMs ke saath!

ADK mein hum iska solution dete hain:
- `Sessions` → **Short term memory** ke liye (current baatcheet yaad rahe)
- `Memory` → **Long term memory** ke liye (permanent yaadein)

Agle notebook mein hum `Memory` par focus karenge.

**🤹 Section 2: Session Management**

### 2.2 Session Kya Hai?

#### **📦 Session**

Session ek container hai jo ek continuous conversation ko store karta hai. Ye conversation history, tool interactions aur responses ko chronologically save karta hai. Har session specific user aur agent ke liye hota hai - dusre users ya agents ke saath share nahi hota.

ADK mein, **Session** do cheezon se banta hai:

**📝 Session.Events**:
- Ye conversation ke building blocks hain
- **Examples:**
  - *User Input*: User ka message (text, audio, image, etc.)
  - *Agent Response*: Agent ka jawab
  - *Tool Call*: Agent koi external tool use kare
  - *Tool Output*: Tool se mila data jo agent aage use karega

**{} Session.State**:
- Ye agent ki "scratchpad" hai - temporary storage
- Global `{key, value}` pairs jo pure conversation mein available rehti hain
- Subagents aur tools sab ise access kar sakte hain

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/session-state-and-events.png" width="320" alt="Session state and events">

<!-- ```mermaid
graph TD
    subgraph A["Agentic Application"];
        subgraph U["User"]
            subgraph S1["Session"]
                D1["Session.Events"]
                D2["Session.State"]
            end
        end
    end
``` -->

**🤹 Section 2: Session Management**

### 2.3 Sessions Kaise Manage Karein?

Ek agentic application mein multiple users ho sakte hain, aur har user ke multiple sessions ho sakte hain. Inhe manage karne ke liye ADK do cheezein provide karta hai:

1. **`SessionService`**: Storage layer
   - Sessions create karna, store karna, aur retrieve karna
   - Different storage options (memory, database, cloud)

2. **`Runner`**: Orchestration layer  
   - User aur agent ke beech information flow manage karta hai
   - Automatically conversation history maintain karta hai
   - Peeche se context engineering handle karta hai

**Simple analogy samjhein:**

- **Session** = Ek notebook 📓
- **Events** = Notebook ke andar likhe hue individual entries 📝
- **SessionService** = Notebooks ko store karne wali filing cabinet 🗄️
- **Runner** = Conversation manage karne wala assistant 🤖

### 2.4 Implementing Our First Stateful Agent

Let's build our first stateful agent, that can remember and have constructive conversations. 

ADK offers different types of sessions suitable for different needs. As a start, we'll start with a simple Session Management option (`InMemorySessionService`):

In [7]:
# Step 0: Basic Setup - Identity Define Kar Rahe Hain
APP_NAME = "default"    # Humari Application ka naam
USER_ID = "default"     # User ki pehchaan  
SESSION = "default"     # Conversation session

MODEL_NAME = "gemini-2.5-flash-lite"  # Kaunsi AI model use karenge

# Step 1: LLM Agent Banayein
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Ye agent kya kaam karega
)

# Step 2: Session Management Setup
# InMemorySessionService - Conversations temporary RAM mein store karega
session_service = InMemorySessionService()

# Step 3: Runner Banayein (Orchestration Layer)
# Ye saari cheezein connect karega aur manage karega
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent taiyar hai!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Storage Type: {session_service.__class__.__name__}")

✅ Stateful agent taiyar hai!
   - Application: default
   - User: default
   - Storage Type: InMemorySessionService


### 2.5 Testing Our Stateful Agent

Now let's see the magic of sessions in action!

In [8]:
# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of United States?",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, I am Sam! What is the capital of United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


🎉 **Success!** The agent remembered your name because both queries were part of the same session. The Runner automatically maintained the conversation history.

But there's a catch: `InMemorySessionService` is temporary. **Once the application stops, all conversation history is lost.** 


### 🛑 (Optional) 2.6 Testing Agent's forgetfulness

> To verify that the agent forgets the conversation, **restart the kernel**. Then, **run ALL the previous cells in this notebook EXCEPT the `run_session` in 2.5.**
> 
> Now run the cell below. You'll see that the agent doesn't remember anything from the previous conversation.

In [9]:
# Run this cell after restarting the kernel. All this history will be gone...
await run_session(
    runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",
)  # Note, we are using same session name


 ### Session: stateful-agentic-session

User > What did I ask you about earlier?
gemini-2.5-flash-lite >  You asked me about the capital of the United States.

User > And remind me, what's my name?
gemini-2.5-flash-lite >  Your name is Sam!


### The Problem

Session information is not persistent (i.e., meaningful conversations are lost). While this is advantageous in testing environments, **in the real world, a user should be able to refer from past and resume conversations.** To achieve this, we must persist information. 

---
## 📈 Section 3: Persistent Sessions with `DatabaseSessionService`

While `InMemorySessionService` is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

### 3.1 Choosing the Right SessionService

ADK provides different SessionService implementations for different needs:

| Service | Use Case | Persistence | Best For |
|---------|----------|-------------|----------|
| **InMemorySessionService** | Development & Testing | ❌ Lost on restart | Quick prototypes |
| **DatabaseSessionService** | Self-managed apps | ✅ Survives restarts | Small to medium apps |
| **Agent Engine Sessions** | Production on GCP | ✅ Fully managed | Enterprise scale |


### 3.2 Implementing Persistent Sessions

Let's upgrade to `DatabaseSessionService` using SQLite. This gives us persistence without needing a separate database server for this demo.

Let's create a `chatbot_agent` capable of having a conversation with the user.

In [10]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


### 3.3 Test Run 1: Verifying Persistence

In this first test run, we'll start a new conversation with the session ID `test-db-session-01`. We will first introduce our name as 'Sam' and then ask a question. In the second turn, we will ask the agent for our name.

Since we are using `DatabaseSessionService`, the agent should remember the name.

After the conversation, we'll inspect the `my_agent_data.db` SQLite database directly to see how the conversation `events` (the user queries and model responses) are stored.


In [11]:
await run_session(
    runner,
    ["Hi, I am Sam! What is the capital of the United States?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > Hi, I am Sam! What is the capital of the United States?
gemini-2.5-flash-lite >  Hello Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam.


### 🛑 (Optional) 3.4 Test Run 2: Resuming a Conversation

> ‼️ Now, let's repeat the test again, but this time, **let's stop this Kaggle Notebook's kernel and restart it again.**
>
> 1. Run all the previous cells in the notebook, **EXCEPT** the previous Section 3.3 (`run_session` cell).
>
> 2. Now, run the below cell with the **same session ID** (`test-db-session-01`).

We will ask a new question and then ask for our name again. **Because the session is loaded from the database, the agent should still remember** that our name is 'Sam' from the first test run. This demonstrates the power of persistent sessions.


In [12]:
await run_session(
    runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > What is the capital of India?
gemini-2.5-flash-lite >  The capital of India is New Delhi.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam.


### 3.5 Let's verify that the session data is isolated

As mentioned earlier, a session is private conversation between an Agent and a User (i.e., two sessions do not share information). Let's run our `run_session` with a different session name `test-db-session-02` to confirm this.


In [13]:
await run_session(
    runner, ["Hello! What is my name?"], "test-db-session-02"
)  # Note, we are using new session name


 ### Session: test-db-session-02

User > Hello! What is my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, including your name. I am a language model and do not have the ability to store or retrieve personal data about users.


### 3.6 How are the events stored in the Database?

Since we are using a sqlite DB to store information, let's have a quick peek to see how information is stored.

In [14]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hello Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "What is the capital of India?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "The capital of India is New Delhi."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text

---
## ⏳ Section 4: Context Compaction

As you can see, all the events are stored in full in the session Database, and this quickly adds up. For a long, complex task, this list of events can become very large, leading to slower performance and higher costs.

But what if we could automatically summarize the past? Let's use ADK's **Context Compaction** feature to see **how to automatically reduce the context that's stored in the Session.**

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/context-compaction.png" width="1400" alt="Context compaction">

### **Section 4.1: Agent ke liye App Banayein**

Hum Section 3.2 mein banaye hue `chatbot_agent` ko hi use karenge.

Pehla step hai ek `App` object banana. Humein iska naam dena hoga aur apna `chatbot_agent` pass karna hoga.

Saath hi, hum Context Compaction ke liye ek naya config banayenge. Ye **`EventsCompactionConfig`** do important cheezein define karta hai:

- **`compaction_interval`**: Runner ko batata hai ki har `n` conversations ke baad history compact kare
- **`overlap_size`**: Overlap ke liye pichli kitni conversations retain karni hain

Last mein, hum ye app Runner ko provide karenge.

**Simple bhasha mein:**
- **App** = Ek wrapper jo agent ko additional features deta hai
- **Context Compaction** = Purani conversations ko summarize karke memory save karna
- **Compaction Interval** = Kitni baatcheet ke baad summarize kare
- **Overlap** = Kuch purani details retain rakhna taaki context maintain ho

In [15]:
# Naya App Banayein - Events Compaction Ke Saath
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,  # Wohi agent use kar rahe hain
    # Yeh Naya Feature Hai!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Har 3 conversations ke baad compaction karega
        overlap_size=1,  # 1 purani conversation context ke liye retain karega
    ),
)

# Database Connection Setup (Same as Before)
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Naya Runner Banayein - Compact App Ke Saath
research_runner_compacting = Runner(
    app=research_app_compacting, 
    session_service=session_service
)

print("✅ Research App Upgrade Ho Gaya - Events Compaction Active!")

✅ Research App Upgrade Ho Gaya - Events Compaction Active!


/tmp/ipykernel_48/3420240245.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### 4.2 Running the Demo

Now, let's have a conversation that is long enough to trigger the compaction. When you run the cell below, the output will look like a normal conversation. However, because we configured our `App`, a compaction process will run silently in the background after the 3rd invocation.

In the next step, we'll prove that it happened.

In [16]:
# Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of the latest news and trends in AI in healthcare:

**Key Areas of Advancement and News:**

*   **Drug Discovery and Development:**
    *   **Accelerated Discovery:** AI continues to be a game-changer here. Companies are using AI to identify potential drug candidates, predict their efficacy, and optimize clinical trial design, significantly shortening timelines and reducing costs.
    *   **Personalized Medicine:** AI is analyzing vast datasets (genomic, proteomic, clinical) to identify personalized treatment plans for patients, especially in areas like oncology.
    *   **Recent News Highlights:** Look for announcements from major pharmaceutical companies and biotech startups detailing AI-driven breakthroughs in identifying novel targets or developing new therapies for diseases like Alzheimer's, cancer, and infectious diseases.

*   **Diagnostics and Medical

**Section 4.3: Session History mein Compaction Verify Karna**

Upar wali conversation normal lag rahi thi, lekin background mein history change ho gayi hai. Ye kaise prove karein?

Hum apne session ka `events` list inspect kar sakte hain. Compaction process **purane events ko delete nahi karta; unhe ek naye `Event` se replace kar deta hai jo summary contain karta hai.** Aao use dhundhte hain.

**Simple Steps:**
1. Session ki events list access karo
2. Dekho kya koi aisa event hai jiska type "summary" ya "compaction" hai
3. Us event ka content check karo - wahi compressed history hogi

**Example:**
```python
session = await session_service.get_session(...)
for event in session.events:
    if "summary" in event.type:  # Ya koi similar identifier
        print("Yeh dekho compressed history:", event.content)
```

Isse pata chal jayega ki compaction actually hua hai ya nahi! 🔍

In [17]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='4fd77796-bd3c-4304-a56e-eeeb3b553d26' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763533299.880072, 'end_timestamp': 1763533307.478045, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data'

### Section 4.4: Tumne Kya Achieve Kiya - Automatic Context Management

Tumne proof dhundh liya! Session history mein woh special summary `Event` ka milna hi compaction process ka result hai.

**Tumne abhi kya dekha:**

1. **Silent Operation**: Normal conversation chala, bahar se koi farak nahi dikha
2. **Background Compaction**: `EventsCompactionConfig` ki wajah se ADK `Runner` automatically conversation length monitor kar raha tha. Threshold cross hote hi usne background mein summarization shuru kar di
3. **Verified Result**: Session events check karke tumne LLM ka banaya hua summary dhundh liya. Ye summary ab puri lengthy history ki jagah use hoga

**Ab is conversation ke aage ke sabhi turns mein,** agent ko puri history nahi, ye concise summary diya jayega. 

**Benefits:**
- Cost bachta hai
- Performance better hoti hai  
- Agent important cheezon par focus kar pata hai

### Section 4.5: ADK mein other Context Engineering Options

#### 👉 Custom Compaction
Is example mein humne ADK ka default summarizer use kiya. Advanced use cases ke liye, tum apna khud ka `SlidingWindowCompactor` bana sakte ho aur use config mein pass kar sakte ho. Isse tum:
- Summarization prompt customize kar sakte ho
- Alag, specialized LLM use kar sakte ho

#### 👉 Context Caching  
ADK **Context Caching** bhi provide karta hai - ye LLM ko diye jaane wale static instructions ke token size ko reduce karta hai by request data cache karke.

**Simple Matlab:**
- **Custom Compaction** = Apna tarika se summary banwana
- **Context Caching** = Repeated instructions ko cache karke performance improve karna

[Official Documentation](https://google.github.io/adk-docs/) mein aur details hai! 📚

### Problem: Cross-Session Information Sharing

Ab tak humne seekha:
- **Context Compaction** → Ek session ki history summarize karna
- **Database Sessions** → Ek session ko resume karna

**Nayi Challenge:** Kuch important information ya preferences hum **multiple sessions mein share** karna chahte hain.

**Example:**
- User ka naam ("Sam")
- Language preference ("Hindi") 
- Theme preference ("Dark mode")

**Problem:** Pure session history share karna efficient nahi hai. Bas kuch key variables transfer karna better hai.

**Solution:** ADK mein **Memory** feature use karenge jo cross-session data sharing enable karta hai! 🚀

### Section 5: Session State ke Saath Kaam Karna

### 5.1 Session State Manage karne ke liye Custom Tools Banayein

Aao manually session state manage karna seekhein custom tools ke through. Is example mein, hum ek **transferable characteristic** identify karenge - jaise user ka naam aur country - aur use save/retrieve karne ke liye tools banayenge.

**Yeh Example Kyu Achha Hai?**

Username ek perfect example hai aisi information ka jo:
- Ek baar introduce hoti hai lekin baar-baar reference hoti hai
- Pure conversation mein persist karni chahiye
- User-specific characteristic hai jo personalization improve karti hai

**Yahan Demo ke Liye:** Hum do tools banayenge:
1. User ka naam aur country store karne ke liye
2. User ka naam aur country retrieve karne ke liye

**Important Note:** Saare tools ko `ToolContext` object access kar sakte hain. Har information ke liye alag tools banane ki zaroorat nahi hai.

In [18]:
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name and country from session state.
    """
    # Read from session state
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("✅ Tools created.")

✅ Tools created.


**Key Concepts:**
- Tools can access `tool_context.state` to read/write session state
- Use descriptive key prefixes (`user:`, `app:`, `temp:`) for organization
- State persists across conversation turns within the same session

### 5.2 Creating an Agent with Session State Tools

Now let's create a new agent that has access to our session state management tools:

In [19]:
# Configuration
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool. 
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo, retrieve_userinfo],  # Provide the tools to the agent
)

# Set up session service and runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("✅ Agent with session state tools initialized!")

✅ Agent with session state tools initialized!


### 5.3 Testing Session State in Action

Let's test how the agent uses session state to remember information across conversation turns:

In [20]:
# Test conversation demonstrating session state
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Sam. I'm from Poland.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing well, thank you for asking. I can't recall your name at the moment. Could you please tell me what it is?

User > My name is Sam. I'm from Poland.


gemini-2.5-flash-lite >  It's nice to meet you, Sam! I'll remember that you're from Poland.

User > What is my name? Which country am I from?


gemini-2.5-flash-lite >  Your name is Sam and you are from Poland.


### 5.4 Inspecting Session State

Let's directly inspect the session state to see what's stored:

In [21]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("Session State Contents:")
print(session.state)
print("\n🔍 Notice the 'user:name' and 'user:country' keys storing our data!")

Session State Contents:
{'user:name': 'Sam', 'user:country': 'Poland'}

🔍 Notice the 'user:name' and 'user:country' keys storing our data!


### 5.5 Session State Isolation

As we've already seen, an important characteristic of session state is that it's isolated per session. Let's demonstrate this by starting a new session:

In [22]:
# Start a completely new session - the agent won't know our name
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)

# Expected: The agent won't know the name because this is a different session


 ### Session: new-isolated-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great. I'm not sure what your name is, though. Can you tell me? 



### 5.6 Cross-Session State Sharing

While sessions are isolated by default, you might notice something interesting. Let's check the state of our new session (`new-isolated-session`):

In [23]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("New Session State:")
print(session.state)

# Note: Depending on implementation, you might see shared state here.
# This is where the distinction between session-specific and user-specific state becomes important.

New Session State:
{'user:name': 'Sam', 'user:country': 'Poland'}


---

## 🧹 Cleanup

In [24]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

✅ Cleaned up old database files


---
## 📊 Summary

🎉 Congratulations! You've learned the fundamentals of building stateful AI agents:

- ✅ **Context Engineering** - You understand how to assemble context for LLMs using Context Compaction
- ✅ **Sessions & Events** - You can maintain conversation history across multiple turns
- ✅ **Persistent Storage** - You know how to make conversations survive restarts
- ✅ **Session State** - You can track structured data during conversations
- ✅ **Manual State Management** - You've experienced both the power and limitations of manual approaches
- ✅ **Production Considerations** - You're ready to handle real-world challenges


---

## ✅ Congratulations! You did it 🎉

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

Refer to the following documentation to learn more:

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK Sessions](https://google.github.io/adk-docs/)
- [ADK Session-State](https://medium.com/google-cloud/2-minute-adk-manage-context-efficiently-with-artifacts-6fcc6683d274)
- [ADK Session Compaction](https://google.github.io/adk-docs/context/compaction/#define-compactor)

### 🎯 Next Steps - Long Term Memory Systems (Part 2)

#### Why do we need memory?
In this notebook, we manually identified a couple characteristic (username and country) and built tools to manage it. But real conversations involve hundreds of such characteristics:
- User preferences and habits
- Past interactions and their outcomes
- Domain knowledge and expertise levels
- Communication styles and patterns
- Contextual relationships between topics

**The Memory System in ADK automates this entire process**, making it a valuable asset for building truly Context-Aware Agents that can accommodate any user's current and future needs.

In the next notebook (Part 2: Memory Management), you'll learn how to:
- Enable automatic memory extraction from conversations
- Build agents that learn and adapt over time
- Create truly personalized experiences at scale
- Manage long-term knowledge across sessions

Ready to transform your manual state management into an intelligent, automated Memory system? Let's continue to Part 2!

---

| Authors |
| --- |
| [Sampath M](https://www.linkedin.com/in/msampathkumar/) |